In [94]:
import pandas as pd

In [95]:
df_enc = pd.read_csv('data\encounter-events.csv')
df_loit = pd.read_csv('data\loitering-events.csv')
df_ves = pd.read_csv('data/transshipment-vessels.csv')

# Data Transformation

## Preprocessing

### Encounter Dataset

This code:

* removes all entries with duplicate values over all features
* removes outliers in duration_hr column
* transforms timestamps into floats and creates new time features as datetime objects
* adds a support feature to the dataset

In [96]:
df_enc_mod = df_enc.copy()

df_enc_mod = df_enc_mod.loc[df_enc_mod['duration_hr'] <= 65]  # remove outliers detected in the EDA file

df_enc_mod.drop_duplicates(inplace = True)  # remove duplicates

df_enc_mod = df_enc_mod[df_enc_mod['fishing_vessel_mmsi'].astype(int).astype(str).str.len() == 9]
# remove all instances with wrong fishing_vessel_mmsi format

print('{} encounterings left in the dataset'.format(df_enc_mod.shape[0]))

11458 encounterings left in the dataset


In [97]:
df_enc_mod['start_time'] = pd.to_datetime(df_enc['start_time'])  # parse time objects
df_enc_mod['end_time'] = pd.to_datetime(df_enc['end_time'])

df_enc_mod['starting_timestamp'] = df_enc_mod['start_time'].apply(lambda x: x.timestamp()) # Changing format of timestamps from object to float
df_enc_mod['ending_timestamp'] = df_enc_mod['end_time'].apply(lambda x: x.timestamp())

# df_enc_mod.head()

In [98]:
df_enc_mod['from_encounter'] = 1  # support variable need later after merging

In [99]:
# df_enc_mod.info()

### Loitering Dataset

This code:

* removes all entries with duplicate values over all features
* remove outliers in total_even_duration column
* Creates new feature and removes redundant features
* transforms timestamps into floats and creates new time features as datetime objects
* adds a support feature to the dataset
* both transforms and renames features to those in the encounter dataset

In [100]:
df_loit_mod =  df_loit.copy()

df_loit_mod = df_loit_mod.loc[df_loit_mod['total_event_duration'] <= 200]  # remove outliers detected in the EDA file

df_loit_mod.drop_duplicates(inplace = True)  # dropping duplicates

print('{} loitering events left in the dataset'.format(df_loit_mod.shape[0]))

45919 loitering events left in the dataset


The next step consists of creating mean variable for both latitude and longitude coordinates to match the format of the encounter dataset

In [101]:
df_loit_mod['mean_latitude'] = (df_loit_mod['starting_latitude']+df_loit_mod['ending_latitude'])/2  # transforming locations to mean
df_loit_mod['mean_longitude'] = (df_loit_mod['starting_longitude']+df_loit_mod['ending_longitude'])/2
df_loit_mod.drop(columns=['starting_latitude','ending_latitude','starting_longitude', 'ending_longitude'], inplace=True)  # drop old columns

Now we change the format of the dates columns to both a datetime and a timestamp (float) version

In [102]:
df_loit_mod['start_time'] = pd.to_datetime(df_loit['starting_timestamp'])  # Create new start time column with datetime format
df_loit_mod['end_time'] = pd.to_datetime(df_loit['ending_timestamp'])

df_loit_mod['starting_timestamp'] = df_loit_mod['start_time'].apply(lambda x: x.timestamp()) # Changing format of timestamps from object to float
df_loit_mod['ending_timestamp'] = df_loit_mod['end_time'].apply(lambda x: x.timestamp())

# df_loit_mod.head()

We rename the transshipment column make clear that it's a transshipment VESSEL attribute not one of a possible transshipment itself
and rename the duration column to match the format of the encounter data set

In [103]:
df_loit_mod.rename(columns ={'transshipment_mmsi':'transshipment_vessel_mmsi', 'total_event_duration':'duration_hr'}, inplace = True)

df_loit_mod['from_encounter'] = 0  # support variable need later after merging

In [104]:
# df_loit_mod.info()

### Vessel Dataset

This code:

* removes all entries with duplicate values in mmsi [Team Decision] to avoid issues with future joins over all datasets
* removes all entries with NA or null values in the mmsi feature
* transforms timestamps into datetime objects

In [105]:
df_ves_mod = df_ves.copy()

df_ves_mod.dropna(inplace=True)
# this is necessary to avoid weird (non sql) behavior from merge (nulls are recognized as keys!)

df_ves_mod = df_ves_mod[df_ves_mod['mmsi'].astype(int).astype(str).str.len() == 9]  # get rid of vessel with wrong mmsi format

df_ves_mod.drop_duplicates(subset = ['mmsi'], inplace = True)  # dropping duplicates in feature mmsi

For the dates, we want to have both times in datetime format and timestamps in float format

In [106]:
df_ves_mod['first_time'] = pd.to_datetime(df_ves_mod['first_timestamp'])  # Create new first time column with datetime format
df_ves_mod['last_time'] = pd.to_datetime(df_ves_mod['last_timestamp'])

df_ves_mod['first_timestamp'] = df_ves_mod['first_time'].apply(lambda x: x.timestamp())  # Change format from object to float
df_ves_mod['last_timestamp'] = df_ves_mod['last_time'].apply(lambda x: x.timestamp())

In [107]:
# df_ves_mod.info()

## Aggregation

This code:

* Aggregates(Stacks) Encounter and Loitering Datasets by keys and adds non matching features as additional columns
* Reinserts the target feature as the first column 

In [108]:
df_agg = pd.concat([df_enc_mod,df_loit_mod], ignore_index=True)

#setting support variable as first column:
df_agg.insert(0, 'from_encounter',df_agg.pop('from_encounter'))

# df_agg.info()

##  Merging

This code:

* Merges the aggregated datasets with the vessel data on the key pair 'transshipment_vessel_mmsi' and 'mmsi'
* This is sufficient because no fishing vessels are in the vessel dataset
* manually adds a suffix of '_t' to all features originating from the vessel dataset


In [109]:
df_clean_all = pd.merge(df_agg, df_ves_mod, how ='left', left_on ='transshipment_vessel_mmsi', right_on='mmsi')


####Optional: add suffix to all added variables
df_clean_all.rename(columns= { 'mmsi' : 'mmsi_t', 'shipname': 'shipname_t', 'callsign':'callsign_t', 'flag':'flag_t', 'imo': 'imo_t',\
     'first_timestamp': 'first_timestamp_t', 'last_timestamp':'last_timestamp_t'}, inplace = True)

####This could be use to merge on fishing vessel, this is however not necessary in this context(No fishing vessels, only tranship)

# df_clean_all = pd.merge(df_clean_all, df_ves_mod, how ='left', left_on ='fishing_vessel_mmsi', right_on='mmsi')
# df_clean_all.rename(columns= { 'mmsi' : 'mmsi_f', 'shipname': 'shipname_f', 'callsign':'callsign_f', 'flag':'flag_f', 'imo': 'imo_f',\
#      'first_timestamp': 'first_timestamp_f', 'last_timestamp':'last_timestamp_f'}, inplace = True)

df_clean_all.dropna(subset=['imo_t'],inplace=True) # some transshipment vessels apparently do not have an ímo code, those have to be removed

In [110]:
df_clean_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50401 entries, 0 to 57376
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   from_encounter             50401 non-null  int64              
 1   fishing_vessel_mmsi        10126 non-null  float64            
 2   transshipment_vessel_mmsi  50401 non-null  int64              
 3   start_time                 50401 non-null  datetime64[ns, UTC]
 4   end_time                   50401 non-null  datetime64[ns, UTC]
 5   mean_latitude              50401 non-null  float64            
 6   mean_longitude             50401 non-null  float64            
 7   duration_hr                50401 non-null  float64            
 8   median_distance_km         10126 non-null  float64            
 9   median_speed_knots         50401 non-null  float64            
 10  starting_timestamp         50401 non-null  float64            
 11  en

In [111]:
df_clean_all

,from_encounter,fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,...,ending_timestamp,mmsi_t,shipname_t,callsign_t,flag_t,imo_t,first_timestamp_t,last_timestamp_t,first_time,last_time
0,1,416565000.0,354240000,2016-11-18 14:30:00+00:00,2016-11-19 01:50:00+00:00,-17.039085,-79.063725,11.333333,0.038188,0.585402,...,1.479520e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
1,1,412679190.0,354240000,2016-12-11 14:50:00+00:00,2016-12-11 19:50:00+00:00,-20.269608,-79.244953,5.000000,0.020033,0.575663,...,1.481486e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
2,1,440863000.0,354240000,2017-06-13 12:50:00+00:00,2017-06-15 01:20:00+00:00,-62.640767,-60.690240,36.500000,0.054992,0.019775,...,1.497490e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
3,1,416563000.0,354240000,2016-11-15 11:30:00+00:00,2016-11-16 04:00:00+00:00,-17.046586,-79.061923,16.500000,0.036427,1.023917,...,1.479269e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
4,1,441309000.0,354240000,2017-05-19 00:40:00+00:00,2017-05-19 20:50:00+00:00,-46.627878,-60.554922,20.166667,0.034053,0.544031,...,1.495227e+09,354240000.0,CRYSTAL REEFER,3FFY5,PAN,9017276.0,1.422748e+09,1.489109e+09,2015-01-31 23:43:15+00:00,2017-03-10 01:21:55+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57372,0,NaN,273349630,2017-09-29 07:55:39+00:00,2017-09-29 22:49:11+00:00,51.945812,155.562330,16.982917,NaN,1.092219,...,1.506725e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57373,0,NaN,273349630,2016-11-19 08:08:13+00:00,2016-11-20 00:58:25+00:00,48.793993,141.242218,17.053056,NaN,0.764958,...,1.479604e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57374,0,NaN,273349630,2014-06-27 01:32:36+00:00,2014-06-28 01:21:20+00:00,58.224653,153.102242,25.157083,NaN,0.560717,...,1.403918e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00
57375,0,NaN,273349630,2016-10-29 17:22:12+00:00,2016-10-30 21:12:23+00:00,57.452946,150.443527,29.134444,NaN,0.545175,...,1.477862e+09,273349630.0,ZODIAK,UBFG9,RUS,8712300.0,1.328268e+09,1.530318e+09,2012-02-03 11:26:34+00:00,2018-06-30 00:13:21+00:00


## Output
Now we are saving the dataset that we are going to use for clustering in the next step.

In [112]:
df_clean_all.to_csv('data/cleaned_data.csv', index=False)